# Prompt Engineering - Notebook exploratoria

Este notebook implementa una **arquitectura RAG básica** para experimentar con diferentes técnicas de prompt engineering en el contexto de una fintech.

## Objetivo
Construir un sistema RAG simple que permita probar y comparar distintos enfoques de prompting sobre un documento de información financiera.

## Secciones del notebook:

1. **Cargar y procesar documento** - Procesamiento del PDF y división en chunks
2. **Crear índice FAISS** - Vector store para búsqueda semántica  
3. **Configurar LLM** - Setup del modelo de lenguaje (GPT-3.5)
4. **Función de consulta RAG** - Pipeline básico de recuperación y generación
5. **Testing de Prompts** - Experimentación con diferentes técnicas:
   - Prompt básico
   - Prompt con contexto
   - Prompt con rol específico  
   - One-shot learning
   - Chain-of-thought
6. **Comparación rápida** - Primeras impresiones de diferencias y qué estaría aportando cada prompt

---


In [1]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
from pypdf import PdfReader
import pickle

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key, "⚠️ Configura OPENAI_API_KEY en tu .env"


## 1. Cargar y procesar documento

In [2]:
file_path = "../01_mock_fintech_info.pdf"

def load_document(file_path):
    if file_path.endswith('.pdf'):
        reader = PdfReader(file_path)
        text = "\n".join([page.extract_text() for page in reader.pages])
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    return text


text = load_document(file_path)


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(text)
documents = [Document(page_content=chunk) for chunk in chunks]

with open("../documents.pkl", "wb") as f:
    pickle.dump(documents, f)

print(f"📄 Documento dividido en {len(chunks)} chunks")

📄 Documento dividido en 33 chunks


## 2. Crear índice FAISS

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ Índice FAISS creado")

✅ Índice FAISS creado


## 3. Configurar LLM

In [4]:
llm = ChatOpenAI(
    temperature=0.3,
    openai_api_key=openai_api_key,
    model="gpt-3.5-turbo"
)

print("🤖 LLM configurado")

🤖 LLM configurado


## 4. Función de consulta RAG

In [5]:
def rag_query(question, custom_prompt=None, k=3):
    """Realiza consulta RAG con prompt."""
    
    docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    default_prompt = """Responde la pregunta.


                    Pregunta: {question}

                    Respuesta:"""
    
    prompt_template = custom_prompt or default_prompt
    prompt = prompt_template.format(context=context, question=question)
    
    response = llm.predict(prompt)
    return response

print("🔍 Función RAG lista")

🔍 Función RAG lista


## 5. Testing de Prompts

### Pregunta de ejemplo

In [6]:
question = "¿Cómo puedo activar mi tarjeta?"

### Prompt Básico

In [7]:
print("=== PROMPT BÁSICO ===")
response1 = rag_query(question)
print(response1)

=== PROMPT BÁSICO ===


/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_80127/1989793315.py:17: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


Para activar tu tarjeta, generalmente puedes hacerlo llamando al número de teléfono que se encuentra en la parte trasera de la tarjeta o ingresando a la página web del emisor de la tarjeta e ingresando la información solicitada para activarla. También puedes activarla en un cajero automático utilizando tu NIP. Es importante seguir las instrucciones específicas proporcionadas por el emisor de la tarjeta para asegurarte de que esté activada correctamente.


### Prompt Básico c/ Contexto

In [8]:
role_prompt = """Responde la pregunta basándote en el contexto.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

print("=== PROMPT CON Contexto Básico ===")
response2 = rag_query(question, role_prompt)
print(response2)

=== PROMPT CON Contexto Básico ===
Puedes activar tu tarjeta de tres maneras: desde la app móvil ingresando a la sección 'Tarjetas', en cajeros automáticos de las redes Banelco o Link, o llamando al servicio al cliente. El método más rápido y seguro es a través de la app móvil.


### Prompt con Rol Específico

In [9]:
role_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech. 
Responde con claridad, precisión y empatía.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

print("=== PROMPT CON ROL ===")
response2 = rag_query(question, role_prompt)
print(response2)

=== PROMPT CON ROL ===
Puedes activar tu tarjeta de tres maneras diferentes. La forma más rápida y segura es a través de la app móvil. Solo tienes que ingresar a la sección 'Tarjetas', seleccionar 'Activar tarjeta' y completar los datos requeridos, como los últimos 4 dígitos de la tarjeta. También puedes activarla en cajeros automáticos de las redes Banelco o Link, insertando la tarjeta y siguiendo el menú 'Gestión de Tarjetas'. Este método es útil si no tienes acceso temporal a la app. Recuerda no compartir tu PIN o CVV y activar las notificaciones para estar al tanto de tus movimientos. ¡Espero que esta información te sea útil! ¿Hay algo más en lo que pueda ayudarte?


### Prompt con One-Shot Example

In [10]:
oneshot_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech.
Siempre responde con claridad, precisión y empatía sobre todo.

<< Ejemplo Pregunta >>
'Estoy evaluando adquirir una tarjeta de crédito, pero tengo dudas sobre sus condiciones y límites.'
<< Fin Ejemplo Pregunta >>

<< Ejemplo Respuesta >>
'¡Hola! Para tu nueva tarjeta de crédito, las condiciones se establecen según tu perfil financiero, lo que determina un límite adecuado para ti. Además, esta tarjeta ofrece beneficios como acumulación de puntos y descuentos en comercios asociados. Puedes revisar más detalles en el estado de cuenta de nuestra app o contactarnos para asesorarte.'
<< Fin Ejemplo Respuesta >>


Debes utilizar el siguiente contexto para responder la consulta:
<< CONTEXTO >>
{context}
<< FIN DEL CONTEXTO >>


La consulta del usuario es la siguiente:
'{question}'

Tu respuesta:
"""

print("=== PROMPT CON ONE-SHOT ===")
response3 = rag_query(question, oneshot_prompt)
print(response3)

=== PROMPT CON ONE-SHOT ===
¡Hola! Para activar tu tarjeta, puedes hacerlo de tres maneras:
1. Desde la app móvil: ingresa a la sección 'Tarjetas', selecciona 'Activar tarjeta' y completa los datos requeridos, como los últimos 4 dígitos de la tarjeta. Este método es rápido y seguro.
2. En cajeros automáticos de las redes Banelco o Link: inserta la tarjeta y sigue el menú 'Gestión de Tarjetas'. Esta opción es útil si no tienes acceso temporal a la app. Recuerda no compartir tu PIN o CVV y activar las notificaciones para estar al tanto de tus movimientos. ¡Espero que esta información te sea útil!


### Prompt con Chain-of-Thought

In [11]:
cot_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech. 
Respondes siempre con claridad, precisión y empatía. Tu objetivo es brindar información útil y confiable, adaptada al perfil del usuario.

<< EJEMPLO DE CONSULTA >>
Estoy evaluando adquirir una tarjeta de crédito, pero tengo dudas sobre sus condiciones y límites.
<< FIN DE CONSULTA >>

<< EJEMPLO DE RESPUESTA >>
¡Hola! Gracias por tu interés en nuestra tarjeta de crédito. Las condiciones varían según tu perfil financiero, lo cual también determina tu límite de crédito. Además, esta tarjeta ofrece beneficios como acumulación de puntos y descuentos en comercios asociados. Para más detalles, revisa tu estado de cuenta en la app o contáctanos y con gusto te asesoraremos.
<< FIN DE RESPUESTA >>

--- CONTEXTO DISPONIBLE ---
{context}
--- FIN DEL CONTEXTO ---

--- INSTRUCCIONES PARA RESPONDER ---
1. Identifica si la consulta es sobre tarjeta de débito, tarjeta de crédito o préstamo.
2. Extrae del contexto los beneficios, condiciones o requisitos más relevantes.
3. Redacta una respuesta clara, empática y estructurada que resuelva la duda del usuario.
--- FIN DE INSTRUCCIONES ---

--- CONSULTA DEL USUARIO ---
{question}
--- FIN DE CONSULTA ---

--- RAZONAMIENTO PASO A PASO ---
1. 
2. 
3. 
--- FIN DE RAZONAMIENTO ---

--- RESPUESTA FINAL ---
"""


print("=== PROMPT CON CHAIN-OF-THOUGHT ===")
response4 = rag_query(question, cot_prompt)
print(response4)

=== PROMPT CON CHAIN-OF-THOUGHT ===
¡Hola! Para activar tu tarjeta, puedes hacerlo de tres maneras:
1. Desde la app móvil: ingresa a la sección 'Tarjetas', selecciona 'Activar tarjeta' y completa los datos requeridos, como los últimos 4 dígitos de la tarjeta. Este método es rápido y seguro.
2. En cajeros automáticos de las redes Banelco o Link: inserta la tarjeta y sigue el menú 'Gestión de Tarjetas'. Es útil si no tienes acceso temporal a la app.
Recuerda no compartir tu PIN o CVV, y activar las notificaciones para estar al tanto de tus movimientos. ¡Espero que esta información te sea útil!


## 5. Comparación rápida

In [12]:
question = "Cuántos dolares puedo extraer por día? Por ejemplo, si quiero extraer 500 dolares puedo? Me quedan 500 dolares netos? Si se cobra comisión, de cuánto es, y en cuyo caso, cuánto debería sacar para que me queden 500 dólares netos?"

In [ ]:
def compare_prompts(question, prompts_dict):
    """Compara diferentes prompts para la misma pregunta."""
    for name, prompt in prompts_dict.items():
        print(f"\n{'='*20} {name.upper()} {'='*20}")
        response = rag_query(question, prompt)
        print(response)
        print("\n" + "-"*60)


prompts = {
    "básico": None,
    "con rol": role_prompt,
    "one shot": oneshot_prompt,
    "combinado": cot_prompt
}

compare_prompts(question, prompts)


==================== BÁSICO ====================
La cantidad de dólares que puedes extraer por día depende de las políticas de tu banco o entidad financiera. Si se cobra una comisión por la transacción, esta se restará del monto total que deseas retirar. Por lo tanto, si quieres que te queden 500 dólares netos y hay una comisión, deberías calcular cuánto debes retirar para que, después de descontar la comisión, te queden 500 dólares.

------------------------------------------------------------

==================== CON ROL ====================
El límite diario para extracciones en cajeros internacionales es de USD 500, con una comisión por operación que deberás tener en cuenta. Si deseas retirar exactamente 500 dólares netos, te recomendamos considerar el monto de la comisión al momento de realizar la operación. Por ejemplo, si la comisión es de USD 5, deberías extraer un total de USD 505 para que te queden 500 dólares netos. Recuerda que es importante realizar extracciones en lugare


<br>

---

<br>



## Observaciones preliminares

**Diferencias observadas en las respuestas:**

- **Prompt básico:** Respuesta genérica sin información específica del documento (no menciona el límite de USD 500). Mucho sentido no tiene claramente porque no tiene información sobre la cual generar la respuesta.
- **Con rol:** Habiendo agregado el contexto y especificando el rol, en este caso aporta información específica (USD 500 límite) + tono empático (gracias al rol) + consejos de seguridad + ejemplo numérico concreto (USD 5 comisión)
- **One-shot:** Sumando al rol, un ejemplo de respuesta, en este caso se ve una mejora en la forma en la que se dirige al usuario (saludo incial, saludo final, e incluso en las expresiones que usa) Información específica + tono amigable, pero menos detallado que "con rol"
- **Chain-of-thought:** Similar contenido al one-shot, aunque en este caso no evidencia el razonamiento paso a paso esperado

**Patrón clave observado:**
- Solo el prompt básico falló en extraer información específica del documento
- Los otros tres prompts lograron identificar correctamente el límite de USD 500 
- El prompt "one shot" fue el más completo: información técnica + ejemplo numérico + consejos adicionales + tono amigable

<br>

---

<br><br>
**Próximo paso:** En la siguiente notebook `02_prompt_evaluation.ipynb` profundizo con evaluación cuantitativa usando LLM-as-a-Judge y RAGAS para medir objetivamente estas diferencias.